# Import

In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

# 한글 폰트 지정
plt.rcParams['font.family'] ='Malgun Gothic'
plt.rcParams['axes.unicode_minus'] =False

font_path = "C:/Windows/Fonts/malgun.ttf"

# Load Data

In [3]:
train_df = pd.read_csv("../../data/train_df_translate_del.csv")
test_df = pd.read_csv("../../data/test_df_translate_del.csv")  
submission = pd.read_csv("../../data/sample_submission.csv")  

In [4]:
print(len(test_df))
print(len(submission))

23405
23405


In [5]:
import pandas as pd

# NaN 값을 찾는 함수
def find_nan_values(df, column_name):
    nan_rows = df[df[column_name].isna()]
    return nan_rows

# train_df와 test_df에서 '키워드' 열의 NaN 값 찾기
nan_train = find_nan_values(train_df, '키워드')
nan_test = find_nan_values(test_df, '키워드')

print("Train DataFrame에서 NaN 값이 있는 행들:")
print(nan_train)

print("\nTest DataFrame에서 NaN 값이 있는 행들:")
print(nan_test)

Train DataFrame에서 NaN 값이 있는 행들:
Empty DataFrame
Columns: [ID, 분류, 제목, 키워드]
Index: []

Test DataFrame에서 NaN 값이 있는 행들:
Empty DataFrame
Columns: [ID, 제목, 키워드]
Index: []


## 전처리

영어로만 구성된 단어에 대한 처리 (주소 제거)

In [6]:
import pandas as pd
import re

# 제거할 단어 리스트
remove_words = ['http', 'www.', '.kr', '.net', '.com']

def remove_specific_words(text):
    words = text.split(',')
    filtered_words = [word for word in words if not any(remove_word in word for remove_word in remove_words)]
    return ','.join(filtered_words)

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(remove_specific_words)
test_df['키워드'] = test_df['키워드'].apply(remove_specific_words)

In [7]:
print(len(test_df))
print(len(submission))

23405
23405


In [8]:
import pandas as pd
import re

# 영어로만 된 단어를 필터링하는 함수
def is_english(word):
    return bool(re.match('^[a-zA-Z]+$', word))

# 영어로만 된 단어를 추출하는 함수
def extract_english_keywords(keywords):
    words = keywords.split(',')
    english_words = [word for word in words if is_english(word)]
    return english_words

# 모든 영어 단어를 하나의 리스트로 모으기
train_english_words = []
test_english_words = []

train_df['키워드'].apply(lambda x: train_english_words.extend(extract_english_keywords(x)))
test_df['키워드'].apply(lambda x: test_english_words.extend(extract_english_keywords(x)))

# 리스트를 시리즈로 변환하여 value_counts 호출
train_english_word_counts = pd.Series(train_english_words).value_counts()
test_english_word_counts = pd.Series(test_english_words).value_counts()

# 2개 이하로 등장하는 단어 제거
train_words_to_remove = train_english_word_counts[train_english_word_counts <= 2].index.tolist()
test_words_to_remove = test_english_word_counts[test_english_word_counts <= 2].index.tolist()

# 원본 데이터에서 해당 단어들을 제거하는 함수
def remove_infrequent_words(keywords, words_to_remove):
    words = keywords.split(',')
    filtered_words = [word for word in words if word not in words_to_remove]
    return ','.join(filtered_words)

# 원본 데이터에 반영
train_df['키워드'] = train_df['키워드'].apply(lambda x: remove_infrequent_words(x, train_words_to_remove))
test_df['키워드'] = test_df['키워드'].apply(lambda x: remove_infrequent_words(x, test_words_to_remove))

# 상위 10개의 결과 출력
filtered_train_english_word_counts = train_english_word_counts[train_english_word_counts > 2]
filtered_test_english_word_counts = test_english_word_counts[test_english_word_counts > 2]

print("Train DataFrame 상위 10개의 결과:")
print(filtered_train_english_word_counts.head(10))

print("\nTest DataFrame 상위 10개의 결과:")
print(filtered_test_english_word_counts.head(10))

Train DataFrame 상위 10개의 결과:
AI       3608
SK       3319
GTX      2956
LH       2919
IC       1842
SNS      1644
CCTV     1532
GH       1510
KLPGA    1410
TF       1386
dtype: int64

Test DataFrame 상위 10개의 결과:
AI       1575
SK       1358
GTX      1275
LH       1255
GH        836
IC        776
SNS       717
TF        619
ESG       613
KLPGA     579
dtype: int64


In [9]:
print(len(test_df))
print(len(submission))

23405
23405


영어, 한글 혼합 단어 처리

In [10]:
import pandas as pd

# 제거할 단어 리스트
remove_words = [
    'http', 'www.', '.kr', '.net', '.com',
    'A씨', 'B씨', 'A양', 'C씨', 'C양', 'D씨', 'A군', 'A사', 'B군', 'B사', 'F팀', 'A업체', 'C노선',
    'B양', 'D등급', 'L자형', 'A의원', 'E씨', 'A그룹', 'A타입', 'C등급', 'B그룹', 'A교사', 'C법인',
    'A선수', 'A법인', 'B등급', 'B노선', 'B타입', 'J씨', 'A시행사', 'A대표', 'G사', 'E등급', 'A경위',
    'B법인', 'R구역', 'A시의원', 'P사', 'K씨', 'C그룹', 'D센터', 'B업체', 'A교회', 'D사', 'H사', 'A아파트',
    'B학생', 'C값', 'F씨', 'S씨', 'C군', 'H공인중개사사무소', 'A캐피탈', 'S등급', 'A고등학교', 'A경사', 'B교회',
    'D법인', 'C사', 'G씨', 'A골프장', 'F노선', 'D노선', 'm가량', 'A팀장', 'L씨', '플랜B', 'B의원', 'A어린이집',
    'S사', 'A농장', 'B지구', 'D구역', 'A산후조리원', 'L사', 'A노래방', 'A사령관', 'Y씨', 'B교사', 'A동', 'C의원',
    'A원장', 'A카페', 'A연구소', 'E주식회사', 'B아파트', 'B어린이집', 'B경장', 'A요양병원', 'F채널', 'A학생',
    'A연습장', 't가량', 'A학원', 'A건설사', 'B동', 'B초등학교', 'B건설사', 'B시장', 'n차', 'p차', 'A식품접객업소',
    'C법', 'B한약국', 'C한약방', 'E군', 't트럭', 'G모사', 'A회사', 'A경찰관', 'A기업', 'B학원', 'C업체',
    'A공인중개사사무소', 'A도시', 'A회장', 'V그룹', 'A고교', 'B동물병원', 'A요양원', 'SA등급', 'B도의원',
    'C변호사', 'B시 B씨네', 'A마트', 'C병원', 'A병원', 'D아파트', 'B공인중개사', 'B선수', 'A협회', 'A사장',
    'D양', 'B개발', 'A조합', 'E노선', '원x2.5', 'A구', 'G그룹', 'S자', 'A공', 'A검사', 'n곤충.kr', 'A팀', '0반',
    'C중학교', 'P차이', 'km가량', 'B조', 'J건설사', 'C컨벤션', 'D건설', 'SS이슈', '시즌A', '시즌B', '시즌C', 'A농협',
    'A약국', 'C약국', 'A구역', '존x', 'A은행', 'kw급', 'D군', 'A헬스장', 'B팀', 'C예식장', 'A단지', 'A지점', 'A지자체',
    't급', 'M교회', 'C블록', 'B목사', '플랜C', 'B번', '개XX', 'D교사', 'B골프장', 'H건설', 'A종합부동산금융사',
    'B세무서장', 'H산업', 'T골프장', 'H씨', 'A강사', 'B강사', 'B원장', 'J기업', 'M타워', 'A현장', 'A소방관', 'C동',
    'F양', 'A초등학생', 'A언론사', 'A당원', 'A시', 'C상병', 'C교수', 'C학원', 'C아파트', 'A등', 'X새끼', 'D구'
]

def remove_exact_words(text):
    words = text.split(',')
    filtered_words = [word for word in words if word not in remove_words]
    return ','.join(filtered_words)

# '키워드' 열에 함수 적용
train_df['키워드'] = train_df['키워드'].apply(remove_exact_words)
test_df['키워드'] = test_df['키워드'].apply(remove_exact_words)

In [11]:
print(len(test_df))
print(len(submission))

23405
23405


In [12]:
import pandas as pd
import re

# 영어와 한글이 혼합된 단어를 필터링하는 함수
def is_mixed_korean_english(word):
    return bool(re.search('[a-zA-Z]', word) and re.search('[가-힣]', word))

# 영어와 한글이 혼합된 단어를 추출하는 함수
def extract_mixed_keywords(keywords):
    words = keywords.split(',')
    mixed_words = [word for word in words if is_mixed_korean_english(word)]
    return mixed_words

# 모든 영어와 한글이 혼합된 단어를 하나의 리스트로 모으기
train_mixed_words = []
test_mixed_words = []

train_df['키워드'].apply(lambda x: train_mixed_words.extend(extract_mixed_keywords(x)))
test_df['키워드'].apply(lambda x: test_mixed_words.extend(extract_mixed_keywords(x)))

# 리스트를 시리즈로 변환하여 value_counts 호출
train_mixed_word_counts = pd.Series(train_mixed_words).value_counts()
test_mixed_word_counts = pd.Series(test_mixed_words).value_counts()

# 3개 이하로 등장하는 단어 제거
train_words_to_remove = train_mixed_word_counts[train_mixed_word_counts <= 3].index.tolist()
test_words_to_remove = test_mixed_word_counts[test_mixed_word_counts <= 3].index.tolist()

# 원본 데이터에서 해당 단어들을 제거하는 함수
def remove_infrequent_words(keywords, words_to_remove):
    words = keywords.split(',')
    filtered_words = [word for word in words if word not in words_to_remove]
    return ','.join(filtered_words)

# 원본 데이터에 반영
train_df['키워드'] = train_df['키워드'].apply(lambda x: remove_infrequent_words(x, train_words_to_remove))
test_df['키워드'] = test_df['키워드'].apply(lambda x: remove_infrequent_words(x, test_words_to_remove))

# 상위 10개의 결과 출력
filtered_train_mixed_word_counts = train_mixed_word_counts[train_mixed_word_counts > 3]
filtered_test_mixed_word_counts = test_mixed_word_counts[test_mixed_word_counts > 3]

print("Train DataFrame 상위 10개의 결과:")
print(filtered_train_mixed_word_counts.head(10))

print("\nTest DataFrame 상위 10개의 결과:")
print(filtered_test_mixed_word_counts.head(10))

Train DataFrame 상위 10개의 결과:
SK하이닉스      4057
GS건설         511
DL이앤씨        323
CJ대한통운       273
QR코드         269
D2블록         224
HL디앤아이한라     214
R&D센터        200
LG전자         199
SK에코플랜트      191
dtype: int64

Test DataFrame 상위 10개의 결과:
SK하이닉스      1548
GS건설         257
DL이앤씨        133
CJ대한통운       127
QR코드         114
GTX-A노선      101
D2블록          86
R&D센터         85
HL디앤아이한라      82
A등급           81
dtype: int64


In [13]:
print(len(test_df))
print(len(submission))

23405
23405


In [14]:
import re
from collections import Counter

# 불용어 제거
removed_keywords = []

def remove_invalid_keywords(keywords):
    pattern = re.compile(
        r'[0-9]+[가-힣\u4E00-\u9FFF]+|'     # 숫자+한글
        r'[0-9]+[A-Za-z]+|'                 # 숫자+영어
        r'[가-힣]+[0-9]+|'                  # 한글+숫자
        r'[A-Za-z]+[0-9]+|'                 # 영어+숫자

        r'[\u4E00-\u9FFF]+|'                # 한자
        r'[0-9]+(\.[0-9]+)?%|'              # 숫자+퍼센트
        r'[0-9]+'                          # 숫자
    )
    valid_keywords = []
    exceptions = ['코로나19', 'RE100', 'GS25']
    for word in keywords.split(','):
        word = word.strip()
        if not word or pattern.match(word):  # 공백이거나 패턴에 맞는 단어 제거
            if word not in exceptions:  # 예외 처리
                removed_keywords.append(word)
            else:
                valid_keywords.append(word)
        else:
            valid_keywords.append(word)
    return ', '.join(valid_keywords)

# 원본 데이터에서 해당 키워드들을 제거
train_df['키워드'] = train_df['키워드'].apply(remove_invalid_keywords)
test_df['키워드'] = test_df['키워드'].apply(remove_invalid_keywords)

# 제거된 단어들 출력
print("\n제거된 단어들:")
print(removed_keywords[:10])


제거된 단어들:
['18일', '15일', '7월', '8월', '50%', '22일', '20명', '10명', '70명', '36종류']


'경기용'과 '전문' 단어 제거

In [15]:
# '경기용'과 '전문' 단어 제거 함수
def remove_keywords(keywords, words_to_remove):
    keywords_list = keywords.split(', ')
    filtered_keywords = [word for word in keywords_list if word not in words_to_remove]
    return ', '.join(filtered_keywords)

# 제거할 단어 목록
words_to_remove = ['경기용', '전문']

# '키워드' 열 업데이트
train_df['키워드'] = train_df['키워드'].apply(lambda x: remove_keywords(x, words_to_remove))
test_df['키워드'] = test_df['키워드'].apply(lambda x: remove_keywords(x, words_to_remove))

In [16]:
# 정확히 일치하는 행 필터링
filtered_df = train_df[train_df['키워드'].str.contains(r'ᅳᆯ', na=False)]

# 결과 출력
filtered_df.iloc[0]['키워드']

'김동성, 양육비, 배드파더스, 등재, 평생, 꼬리표, 인민정, 호소, 김동성, 쇼트, 트랙, 국가, 대표, 극단적, 선택, 인민정, 여자, 친구, 인민정, 자신, SNS, 심경, 인스타그램, 저, ᆼ글맘, 자신, 처지, 공개, 우울증, 공황, 장애, 김동성, 공평, 상황들, 뇌리, 장문, 김동성, ᅥᆫ만원정도, 살아야, 로, 고, 그, 꼬리표는, 평생, 달고, 사, 토로, 김동성씨, 방송, ᅳᆯ, 도, 아무것, ᅡᄀ, 스, 써, 여, ᆷᄒ, ᅩᄅ, 데, 극단적, ᅳᆯ, 할, 수밖에, 어, 바닥, 어, 이, 지옥, 상황, 결과, 호소, 김동성, 지난달, 용인시, 상현동, 자택, 극단적, 선택, 시도, 신고, 출동, 대원, 병원, 이송, 생명, 지장, ᅦᄀ, ᅳᆫ, 질타, 않으면, ᆯᄋ, ᅳᆫ데, 저는, 제가, 눈을, 뜨고, 있는, 한, 어떠한, 상황에서도, 발벗고, 나서지, 아, 일분이, 가, 이, 이, 이, 소중하기ᄋ, ᅳᆫ, 야, 설명, 배드파더스, 경제활동, 악의적, 양육비, 비양육자, 채찍질, 소득, 수입, 재산, 비양육자, 배드파더스, 낙인, 합리적, 개인적, 생각, 아내, 이혼, 양육비, 논란, 김동성, TV조선, 이혼, 프로그램, 출연, 인민정, 인스타, 그램, 싱글맘, 양육자와, 비양육자, ᆼ글맘입니다, 지금껏, 양육비ᄂ, 이엄마입니다, 도, ᅳᆯ, 걸까, 이, 이, 서, 이, 벌었습니다, 도, 아, ᅳᆯ, ᅳᆯ, 김동성씨ᄂ, 고, 랴, 로, 이, ᅥᆫ만원정도입니다, 도, 로, 고, 그, 꼬리표는, 평생, 달고, 사, 남자친구, ᅦᄀ, ᅳᆫ, 고, ᄂ

In [17]:
# 해당 행 제거
train_df = train_df.drop(filtered_df.index)

In [18]:
print(len(test_df))
print(len(submission))

23405
23405


In [31]:
import pandas as pd

# '키워드' 열의 각 문자열 길이 계산
train_df['키워드_길이'] = train_df['키워드'].apply(len)
test_df['키워드_길이'] = test_df['키워드'].apply(len)

# 길이가 긴 순서대로 정렬하여 상위 5개의 길이 출력 (train_df)
top_5_lengths_train = train_df.nlargest(5, '키워드_길이')['키워드_길이']
print("Top 5 키워드 길이 (Train DataFrame):")
print(top_5_lengths_train)

# 길이가 긴 순서대로 정렬하여 상위 5개의 길이 출력 (test_df)
top_5_lengths_test = test_df.nlargest(5, '키워드_길이')['키워드_길이']
print("\nTop 5 키워드 길이 (Test DataFrame):")
print(top_5_lengths_test)

Top 5 키워드 길이 (Train DataFrame):
50050    11637
30489    11568
6135     10764
53513     8899
14412     8688
Name: 키워드_길이, dtype: int64

Top 5 키워드 길이 (Test DataFrame):
19921    12816
20753     9682
7075      8388
16362     7619
2679      7216
Name: 키워드_길이, dtype: int64


In [41]:
# '키워드' 열의 각 문자열에서 쉼표로 구분된 단어의 개수 계산
train_df['단어_개수'] = train_df['키워드'].apply(lambda x: len(x.split(',')))
test_df['단어_개수'] = test_df['키워드'].apply(lambda x: len(x.split(',')))

# 단어 개수를 기준으로 상위 5개의 개수 출력 (train_df)
top_5_word_counts_train = train_df.nlargest(5, '단어_개수')['단어_개수']
print("\nTop 5 단어 개수 (Train DataFrame):")
print(top_5_word_counts_train)

# 단어 개수를 기준으로 상위 5개의 개수 출력 (test_df)
top_5_word_counts_test = test_df.nlargest(5, '단어_개수')['단어_개수']
print("\nTop 5 단어 개수 (Test DataFrame):")
print(top_5_word_counts_test)


Top 5 단어 개수 (Train DataFrame):
50050    2606
30489    2480
14412    1973
53513    1952
53       1767
         ... 
435       330
4259      330
4650      330
5402      330
6449      330
Name: 단어_개수, Length: 3000, dtype: int64

Top 5 단어 개수 (Test DataFrame):
19921    2541
20753    2056
7075     1834
16362    1759
2679     1641
Name: 단어_개수, dtype: int64


In [43]:
# '키워드' 열의 각 문자열에서 쉼표로 구분된 단어의 개수 계산
train_df['단어_개수'] = train_df['키워드'].apply(lambda x: len(x.split(',')))
test_df['단어_개수'] = test_df['키워드'].apply(lambda x: len(x.split(',')))

# 단어 개수를 기준으로 상위 5개의 개수 출력 (train_df)
top_5_word_counts_train = train_df.nlargest(5, '단어_개수')['단어_개수']
print("\nTop 5 단어 개수 (Train DataFrame):")
print(top_5_word_counts_train)

# 단어 개수를 기준으로 상위 5개의 개수 출력 (test_df)
top_5_word_counts_test = test_df.nlargest(5, '단어_개수')['단어_개수']
print("\nTop 5 단어 개수 (Test DataFrame):")
print(top_5_word_counts_test)

# 단어의 개수가 제일 많은 행의 키워드 출력 (train_df)
max_word_count_keyword_train = train_df.loc[train_df['단어_개수'].idxmax(), '키워드']
print("\n단어의 개수가 제일 많은 행의 키워드 (Train DataFrame):")
print(max_word_count_keyword_train)

# 단어의 개수가 제일 많은 행의 키워드 출력 (test_df)
max_word_count_keyword_test = test_df.loc[test_df['단어_개수'].idxmax(), '키워드']
print("\n단어의 개수가 제일 많은 행의 키워드 (Test DataFrame):")
print(max_word_count_keyword_test)


Top 5 단어 개수 (Train DataFrame):
50050    2606
30489    2480
14412    1973
53513    1952
53       1767
Name: 단어_개수, dtype: int64

Top 5 단어 개수 (Test DataFrame):
19921    2541
20753    2056
7075     1834
16362    1759
2679     1641
Name: 단어_개수, dtype: int64

단어의 개수가 제일 많은 행의 키워드 (Train DataFrame):
총체, 난국, 최후, 발악, 안철수, 오세훈, 희망, 집중, 민주당, 땅투기, DNA, 대통령, 보장, 투기, 이익, 대통령, 근심, 걱정, 공수처, 고위공직자범죄꾼, 상담처, 전락, K-방역, 한명숙, 실체, 컴퓨터, 기생충, 단어, 기생충, 학자, 서민, 교수, 사랑, 애정, 긍정, 의미, 기생충, 대한민국, 정치인, 비유, 생물, 통속, 의미, 기생충, 생각, 기생충, 기생충, 국민, 고혈, 흡혈, 기생충, 국민, 등골, 기생충, 국민, 세금, 백수, 기생충, 정도, 차이, 장내, 미생물, 국민들, 도움, 기생충들, 정치인들, 기생충, 폄하, 외면, 기생충, 세계, 정치판, 민낯, 우여곡절, 갈등, 국민, 안철수, 후보, 양보, 선언, 물꼬, 난관, 오세훈, 국민, 후보, 양보, 사람, 국민정서, 이날, 후보, 양보, 선언, 기분, 촌극, 후보, 서울, 시장, 기생충, 정치판, 국민들, 편안, 말뜻, 정치, 실천, 당부, 정치, 바르다, 회초리, 의미, 등글월문, 건축물, 정치, 자신, 사람, 부조화, 부정적, 극복, 대통령, 정도, 매입, 사저, 경남, 양산, 농지, 대지, 형질, 변경, 확인, 페이스북, 문재인, 대통령, 반응, 대통령, 대통령, 본인, 사저, 부지, 투기, 논란, 사과문, 사람, 부부, 문재인, 대통령, 대통령, 가족, 의혹, 부동산, 투기, 분노, 국민들, 제기, 야당, 사과, 합리적, 해명, 국민, 야당, 문재

In [20]:
train_df.drop(columns=['키워드_길이'], inplace=True)
test_df.drop(columns=['키워드_길이'], inplace=True)

In [63]:
# import os
# import pandas as pd
# from datetime import datetime

# # 현재 날짜 가져오기 (MMDD 형식)
# current_date = datetime.now().strftime('%m%d')

# # CSV 파일명 생성
# train_csv_filename = f"train_df_{current_date}.csv"
# test_csv_filename = f"test_df_{current_date}.csv"

# # CSV 파일로 저장
# train_df.to_csv(train_csv_filename, index=False, encoding='utf-8-sig')
# test_df.to_csv(test_csv_filename, index=False, encoding='utf-8-sig')

# print(f"train_df saved as {train_csv_filename}")
# print(f"test_df saved as {test_csv_filename}")

train_df saved as train_df_1006.csv
test_df saved as test_df_1006.csv


.